In [1]:
import numpy as np 
import pandas as pd 

In [ ]:
data = pd.read_excel('sen3.xlsx')
feature = data.iloc[:, 5:-1] 
Y = data['label']
frequency_features = feature.iloc[:,:500]
similarity_features = feature.iloc[:,500:501]
sentiment_features = feature.iloc[:,501:507]
entity_features = feature.iloc[:,507:523]
semantic_features = feature.iloc[:,523:]

In [3]:
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, f1_score, auc,
                              precision_score, recall_score, roc_auc_score, roc_curve, 
                             precision_recall_curve)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [4]:
from itertools import combinations

feature_list = [frequency_features,similarity_features, sentiment_features, entity_features, semantic_features]
combinations_list = []
for i in range(1, len(feature_list) + 1):
    for combination in combinations(feature_list, i):
        combinations_list.append(combination)

In [5]:
from modeleva import *

In [ ]:
for combination in combinations_list:
    print('Combination:', [feature.columns[0] for feature in combination])
    Xcom = pd.concat(combination, axis=1)
    print(Xcom.shape)
    st=StandardScaler()
    Xcom = st.fit_transform(Xcom)
    
    X_train, X_test, y_train, y_test = train_test_split(Xcom, Y, test_size=0.2, random_state=42)
    over_samples = SMOTE(random_state=1234) 
    over_samples_X,over_samples_y = over_samples.fit_resample(X_train, y_train)
    #over_samples_X, over_samples_y = over_samples.fit_sample(X_train.values,y_train.values.ravel())
    # before smote
    print(y_train.value_counts()/len(y_train))
    # after smote
    print(pd.Series(over_samples_y).value_counts()/len(over_samples_y))
    knn = KNeighborsClassifier(n_neighbors=10)
    lr = LogisticRegression()
    gnb = GaussianNB()
    bnb = BernoulliNB()
    tree = DecisionTreeClassifier()
    forest = RandomForestClassifier()
    xgb = XGBClassifier()
    svc = SVC(probability=True)
    model_list = [knn, lr, gnb, bnb, tree, forest, xgb]
    kf = KFold(n_splits=5, shuffle=True, random_state=33)  #5-fold
    multi_model_eval(model_list, over_samples_X,over_samples_y, kf)
    for i in model_list:
        print(accuracy_score(i.predict(X_test),y_test))